In [7]:

def averager():
    count = 0
    total = 0
    average = None
    while True:
        r = yield average
        total += r
        count += 1
        average = total/count

In [8]:
a = averager()
print(type(a))
next(a)

<class 'generator'>


In [3]:
type(a)

generator

In [4]:
a.send(3)

3.0

In [5]:
a.send(5)

4.0

In [6]:
type(a)

generator

In [9]:
a.close()

In [11]:
from collections import namedtuple
result = namedtuple("result","count average")
def averager():
    count = 0
    total = 0
    average = None
    while True:
        r = yield
        if r == None:
            break
        total += r
        count += 1
        average = total/count
    return result(count, average)

In [19]:
a = averager()
next(a)
a.send(10)
a.send(12)
a.send(14)
a.send(16)
a.send(18)
a.send(20)
a.send(22)
a.send(24)
try:
    a.send(None)
except StopIteration as r:
    print(r)

result(count=8, average=17.0)


In [29]:
from collections import namedtuple

Result = namedtuple("result","count average")

def averager():
    count = 0
    total = 0
    average = None
    while True:
        r = yield
        if r == None:
            break
        total += r
        count += 1
        average = total/count
    return Result(count, average)

def grouper(results, key):
    while True:
        results[key] = yield from averager()


data = {
    'girls;kg':
        [40.9, 38.5, 44.3, 42.2, 45.2, 41.7, 44.5, 38.0, 40.6, 44.5],
    'girls;m':
        [1.6, 1.51, 1.4, 1.3, 1.41, 1.39, 1.33, 1.46, 1.45, 1.43],
    'boys;kg':
        [39.0, 40.8, 43.2, 40.8, 43.1, 38.6, 41.4, 40.6, 36.3],
    'boys;m':
        [1.38, 1.5, 1.32, 1.25, 1.37, 1.48, 1.25, 1.49, 1.46],
}

def report(results):
    for key, result in sorted(results.items()):
        group, unit = key.split(';')
        print("{:2} {:5} averaging {:.2f} {}".format(
            result.count, group, result.average, unit
        ))

result = {}
for key, value in data.items():
    group = grouper(result, key)
    next(group)
    for v in value:
        group.send(v)
    group.send(None)
    
report(result)

 9 boys  averaging 40.42 kg
 9 boys  averaging 1.39 m
10 girls averaging 42.04 kg
10 girls averaging 1.43 m


In [ ]:
from collections import namedtuple

Event = namedtuple("Event", "time proc action")

async def taxi_process(ident, trips, start_time=0):
    time = yield Event(start_time, ident, "leave garage") # 出车库
    for i in range(trips):
        time = yield Event(time, ident, "drop off passenger") # 放乘客
        time = yield Event(time, ident, "pick up passenger")  # 接乘客
    yield Event(time, ident, "going home")                  # 回家